In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

--2019-06-11 08:40:33--  http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.216.138.235
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.216.138.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10367709 (9.9M) [application/json]
Saving to: ‘./pw-data/201701scripts_sample.json.gz’

     0K .......... .......... .......... .......... ..........  0% 24.4M 0s
    50K .......... .......... .......... .......... ..........  0% 24.3M 0s
   100K .......... .......... .......... .......... ..........  1% 35.1M 0s
   150K .......... .......... .......... .......... ..........  1% 34.9M 0s
   200K .......... .......... .......... .......... ..........  2% 32.2M 0s
   250K .......... .......... .......... .......... ..........  2% 34.6M 0s
   300K .......... .......... .......... .......... ..........  3% 41.3M 0s
   

## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [3]:
import gzip
import simplejson as json

In [4]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [5]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [6]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [7]:
import math
def describe(key):
    ctr=0
    l=[]
    total=0
    var=0
    for i in scripts:
        l.append(i[key])
        total+=i[key]
        ctr+=1
    avg = total/ctr
    for i in scripts:
        var+=(avg-i[key])**2
    s = (var/ctr)**0.5
    l.sort()
    med = l[math.floor(ctr/2)]
    q25 = l[math.floor(ctr/4)]
    q75 = l[math.floor(ctr*3/4)]

    return (total, avg, s, q25, med, q75)

In [8]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [330]:
print (summary[0][1][0])

4410054


In [9]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [10]:
bnf_names = {}

for i in scripts:
    key=i['bnf_name']
    if key in bnf_names:
        bnf_names[key].append(i)
    else:
        bnf_names[key]=[i]
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [11]:
"""groups = {name: [] for name in bnf_names}
for script in scripts:
    groups[script['bnf_name']].append(script)  """
groups=bnf_names

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [12]:
item=[]
for i in groups.keys():
    sum1=0
    for j in groups[i]:
        sum1+=j['items']
    item.append((sum1,i))
max_item = [max(item)[::-1]]

In [33]:
item[:10]

[(86, 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F'),
 (392, 'Alginate_Raft-Forming Oral Susp S/F'),
 (2636, 'Sod Algin/Pot Bicarb_Susp S/F'),
 (200, 'Sod Alginate/Pot Bicarb_Tab Chble 500mg'),
 (1978, 'Gaviscon Infant_Sach 2g (Dual Pack) S/F'),
 (5568, 'Gaviscon Advance_Liq (Aniseed) (Reckitt)'),
 (1078, 'Gaviscon Advance_Tab Chble Mint(Reckitt)'),
 (3443, 'Gaviscon Advance_Liq (Peppermint) S/F'),
 (3162, 'Peptac_Liq (Peppermint) S/F'),
 (776, 'Alverine Cit_Cap 60mg')]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [13]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [78]:
def group_by_field(data, fields):
    groups_dic={}
    for field in fields:
        groups = {}
        for i in data:
            key=i['%s'%(field)]
            if key in groups:
                groups[key].append(i)
            else:
                groups[key]=[i]
        groups_dic['%s'%(field)]=groups
    return groups_dic

In [79]:
groups = group_by_field(scripts, ('bnf_name',))
item=[]
for k in groups.keys():
    for i in groups[k].keys():
        sum1=0
        for j in groups[k][i]:
            sum1+=j['items']
        item.append((sum1,i))
    test_max_item = [max(item)[::-1]]


assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices'`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [16]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        if (practice_postal[practice['code']]>practice['post_code']):
            practice_postal[practice['code']]=practice['post_code']
    else:
        practice_postal[practice['code']] = practice['post_code']

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [17]:
assert practice_postal['K82019'] == 'HP21 8TR'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [18]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [19]:
joined = scripts[:]
for script in joined:
    if script['practice'] in practice_postal.keys():
        script['post_code'] = practice_postal[script['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [20]:
items_by_post = group_by_field(scripts, ('post_code',))
postal_totals= []
for k in items_by_post.keys():
    for i in items_by_post[k].keys():
        sum1=0
        for j in items_by_post[k][i]:
            sum1+=j['items']
        postal_totals.append((sum1,i))

In [21]:
items_by_post_1 ={}
for i in scripts:
    key=i['post_code']
    if key in items_by_post_1:
        items_by_post_1[key].append(i)
    else:
        items_by_post_1[key]=[i]
groups=items_by_post_1        
postal_total1=[]
for i in groups.keys():
    sum1=0
    for j in groups[i]:
        sum1+=j['items']
    if (sum1)!=0:
        postal_total1.append((i,sum1))

In [22]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        if (practice_postal[practice['code']]>practice['post_code']):
            practice_postal[practice['code']]=practice['post_code']
    else:
        practice_postal[practice['code']] = practice['post_code']

joined = scripts[:]
for script in joined:
    if script['practice'] in practice_postal.keys():
        script['post_code'] = practice_postal[script['practice']]

prescriptionByPostCode = {}
for element in joined:
    if element['post_code'] not in prescriptionByPostCode:
       prescriptionByPostCode[element['post_code']] = [element['items']]
    else:
        prescriptionByPostCode[element['post_code']].append(element['items'])

items_by_post = [sum(prescriptionByPostCode[x]) for x in prescriptionByPostCode]
keys = list(prescriptionByPostCode.keys())
pairs = list(zip(keys, items_by_post))
pairs.sort()
items_by_post = []
for pair in pairs:
    if pair[1] > 0:
        items_by_post.append(pair)            

In [23]:
grader.score.pw__postal_totals(items_by_post[:100])

Your score:  1.0


In [28]:
(postal_total1).sort()
postal_sorted=(postal_total1)[:100]
#print (postal_sorted[:100][::-1])
#postal_total = [(j,i) for i,j in (postal_sorted[::-1][:100])]

grader.score.pw__postal_totals(postal_sorted)

Your score:  1.0


In [334]:
print(items_by_post[:100])
print(postal_sorted)
sum1=0
for i in postal_sorted:
    sum1+=i[1]
print (sum1)

[('B11 4BW', 20673), ('B18 7AL', 19001), ('B21 9RY', 29103), ('B23 6DJ', 24859), ('B70 7AW', 36531), ('BB11 2DL', 34356), ('BB2 1AX', 28254), ('BB3 1PY', 54514), ('BB4 5SL', 29388), ('BB7 2JG', 44585), ('BB8 0JZ', 54380), ('BB9 7SR', 38224), ('BD3 8QH', 21010), ('BH18 8EE', 39413), ('BH23 3AF', 32545), ('BL1 8TU', 26132), ('BL3 5HP', 27147), ('BL9 0NJ', 32062), ('BL9 0SN', 35275), ('CB9 8HF', 51337), ('CH1 4DS', 34915), ('CH65 6TG', 25090), ('CT11 8AD', 44358), ('CV1 4FS', 37210), ('CW1 3AW', 64104), ('CW5 5NX', 38797), ('CW7 1AT', 43164), ('DA1 2HA', 26075), ('DA11 8BZ', 24090), ('DN22 7XF', 43091), ('DN34 4GB', 48013), ('FY2 0JG', 69118), ('FY4 1TJ', 62886), ('FY5 2TZ', 44258), ('FY7 8GU', 34473), ('GL1 3PX', 38120), ('GL50 4DP', 74822), ('GU9 9QS', 32131), ('HA0 4UZ', 22755), ('HA3 7LT', 32113), ('HG1 5AR', 32684), ('HU7 4DW', 49107), ('KT14 6DH', 26758), ('KT6 6EZ', 38975), ('L31 0DJ', 32065), ('L36 7XY', 22965), ('L5 0QW', 24676), ('L7 6HD', 42569), ('LA1 1PN', 47335), ('LE10 1DS'

## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [30]:
joined[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000,
  'post_code': 'SK11 6JL'},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500,
  'post_code': 'SK11 6JL'}]

In [86]:
post_code=group_by_field(joined, ('post_code',))
print (post_code.keys())
print (len(post_code['post_code']))
item=[]
for k in bnf_name.keys():
    sum2=0
    for i in bnf_name[k].keys():
        sum1=0
        for j in bnf_name[k][i]:
            sum1+=j['items']
        item.append((sum1,i))
        sum2+=sum1
    print (sum2)
#bnf_name=group_by_field(joined,('bnf_name',))

dict_keys(['post_code'])
118


NameError: name 'bnf_name' is not defined

In [111]:
bnf_names = {}
for i in joined:
    key=i['bnf_name']
    key2=i['post_code']
    if (key,key2) in bnf_names:
        bnf_names[(key,key2)].append(i)
    else:
        bnf_names[(key,key2)]=[i]
groups=bnf_names
item=[]
for i,j in groups.keys():
    sum1=0
    for k in groups[(i,j)]:
        sum1+=k['items']
    item.append([j,i,sum1])

In [112]:
total_items_by_bnf_post = item
assert len(total_items_by_bnf_post) == 141196

Let's use `total_by_item_post` to find the maximum item total for each postal code. To do this, we will want to regroup `total_by_item_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_by_item_post` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in `total_by_item_post` after grouping it by `'post_code'`.

In [470]:
def group_by_field2(data, fields):
    groups_dic={}
    for field in fields:
        groups = {}
        for i in data:
            key=i['%s'%(field)]
            if key in groups.keys():
                flag=0
                for name in range(len(groups[key])):
                    if (groups[key][name][1]==i['bnf_name']):
                        groups[key][name][0]+=i['items']
                        flag=1
                        break
                if flag==0:
                    groups[key].append([i['items'],i['bnf_name']])
            else:
                groups[key]=[[i['items'],i['bnf_name']]]
        groups_dic['%s'%(field)]=groups
        
    return groups_dic

In [471]:
total_items_by_post = group_by_field2(joined, ('post_code',))['post_code']
assert len(total_items_by_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [474]:
test=total_items_by_post

In [424]:
for key in test:
    test['%s'%key].sort(reverse=True)

In [500]:
anslist=[]
ctr=0
total_items=items_by_post
for key in sorted(list(test.keys())):
    anslist.append((key,'%s'%max(test['%s'%key])[1],(max(test['%s'%key])[0]/total_items[ctr][1])))
    ctr+=1
anslist.sort()
#anslist.append((key,test['%s'%key],max(test['%s'%key][])))----//total_items[ctr][1]

In [496]:
list(test.keys()).sort()

In [484]:
max_item_by_post = anslist[:100]

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [312]:
items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [501]:
items_by_region = anslist[:100]

In [502]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*